In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('wine-reviews/winemag-data_first150k.csv')
# df

In [27]:
# df['description']

In [8]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
# nltk.download('wordnet')
  
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [9]:
doc_clean = [clean(doc).split() for doc in df['description']]   

In [10]:
df['description'] = doc_clean
# df

In [11]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our corpus, where every unique term is assigned an index.
dictionary = corpora.Dictionary(df['description'])

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['description']]

In [33]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix.
ldamodel = LDA(doc_term_matrix, num_topics=3, id2word = dictionary, passes=5)

In [34]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, '0.022*"flavor" + 0.021*"finish" + 0.019*"cherry"'), (1, '0.035*"wine" + 0.023*"fruit" + 0.021*"flavor"'), (2, '0.026*"flavor" + 0.024*"wine" + 0.017*"finish"')]


In [16]:
from gensim.test.utils import common_corpus, get_tmpfile

LSA = gensim.models.LsiModel

lsa = LSA(doc_term_matrix, num_topics=10, id2word=dictionary)  # train model

print(lsa.print_topics(num_topics=10, num_words=10))

[(0, '0.480*"wine" + 0.388*"flavor" + 0.332*"fruit" + 0.190*"finish" + 0.173*"cherry" + 0.164*"aroma" + 0.161*"tannin" + 0.145*"acidity" + 0.126*"palate" + 0.126*"ripe"'), (1, '-0.683*"wine" + 0.421*"flavor" + 0.290*"finish" + -0.197*"fruit" + 0.197*"palate" + 0.185*"aroma" + 0.172*"cherry" + 0.097*"note" + 0.088*"nose" + 0.082*"berry"'), (2, '0.722*"fruit" + -0.411*"flavor" + -0.349*"wine" + 0.146*"black" + 0.125*"tannin" + -0.123*"dry" + 0.108*"palate" + 0.107*"aroma" + 0.093*"note" + 0.079*"finish"'), (3, '0.417*"cherry" + 0.343*"tannin" + 0.293*"black" + -0.200*"apple" + 0.194*"blackberry" + -0.190*"finish" + -0.188*"fruit" + -0.162*"citrus" + -0.153*"acidity" + -0.143*"aroma"'), (4, '0.406*"flavor" + 0.389*"fruit" + -0.368*"aroma" + -0.300*"wine" + -0.275*"palate" + -0.235*"finish" + -0.193*"note" + 0.173*"acidity" + -0.153*"berry" + 0.133*"ripe"'), (5, '-0.466*"acidity" + 0.394*"flavor" + -0.295*"cherry" + -0.239*"note" + 0.238*"aroma" + -0.214*"drink" + -0.163*"crisp" + -0.158*"

In [18]:
results = lsa.print_topics(num_topics=10, num_words=10)

results = [entry[1].split('+') for entry in results]

dataframe = pd.DataFrame(results)
dataframe

,0,1,2,3,4,5,6,7,8,9
0,"0.480*""wine""","0.388*""flavor""","0.332*""fruit""","0.190*""finish""","0.173*""cherry""","0.164*""aroma""","0.161*""tannin""","0.145*""acidity""","0.126*""palate""","0.126*""ripe"""
1,"-0.683*""wine""","0.421*""flavor""","0.290*""finish""","-0.197*""fruit""","0.197*""palate""","0.185*""aroma""","0.172*""cherry""","0.097*""note""","0.088*""nose""","0.082*""berry"""
2,"0.722*""fruit""","-0.411*""flavor""","-0.349*""wine""","0.146*""black""","0.125*""tannin""","-0.123*""dry""","0.108*""palate""","0.107*""aroma""","0.093*""note""","0.079*""finish"""
3,"0.417*""cherry""","0.343*""tannin""","0.293*""black""","-0.200*""apple""","0.194*""blackberry""","-0.190*""finish""","-0.188*""fruit""","-0.162*""citrus""","-0.153*""acidity""","-0.143*""aroma"""
4,"0.406*""flavor""","0.389*""fruit""","-0.368*""aroma""","-0.300*""wine""","-0.275*""palate""","-0.235*""finish""","-0.193*""note""","0.173*""acidity""","-0.153*""berry""","0.133*""ripe"""
5,"-0.466*""acidity""","0.394*""flavor""","-0.295*""cherry""","-0.239*""note""","0.238*""aroma""","-0.214*""drink""","-0.163*""crisp""","-0.158*""fresh""","0.156*""wine""","-0.143*""dry"""
6,"-0.477*""cherry""","0.357*""finish""","0.279*""tannin""","-0.254*""red""","0.229*""ripe""","0.196*""blackberry""","0.194*""note""","-0.186*""fruit""","0.174*""drink""","0.157*""rich"""
7,"-0.446*""acidity""","-0.349*""tannin""","0.293*""finish""","-0.275*""aroma""","-0.261*""berry""","0.237*""note""","0.212*""cherry""","0.175*""oak""","-0.175*""ripe""","0.162*""fruit"""
8,"0.383*""ripe""","-0.345*""finish""","0.338*""sweet""","0.318*""spice""","-0.247*""dry""","0.235*""rich""","-0.214*""tannin""","0.213*""aroma""","0.172*""oak""","-0.162*""red"""
9,"0.388*""black""","-0.337*""sweet""","-0.321*""finish""","-0.228*""ripe""","-0.198*""soft""","-0.197*""berry""","0.195*""dry""","0.172*""cabernet""","0.162*""white""","-0.154*""red"""
